In [16]:
from germanhass.DBCode.HassDBAdapter import HassDBAdapter
from tqdm import tqdm
from multiprocess import Pool
import pandas as pd
from os.path import join
import numpy as np

In [2]:
# connect to the db
db = HassDBAdapter()

In [3]:
test_hass = list(db.hate_tweets_coll\
    .find({"test_set":"dataset1"}, {'full_text':1, 'account_type':1}))
train_hass = list(db.hate_tweets_coll\
    .find({"training_set":"dataset1"}, {'full_text':1, 'account_type':1}))

test_counter = list(db.counter_tweets_coll\
    .find({"test_set":"dataset1"}, {'full_text':1, 'account_type':1}))
train_counter = list(db.counter_tweets_coll\
    .find({"training_set":"dataset1"}, {'full_text':1, 'account_type':1}))

In [4]:
def extract_information(entry):
    row = {
        'id':entry['_id'],
        'text':entry['full_text'],
        'label':entry['account_type']
    }
    return row

In [9]:
dst = '../data/traindata'
number_of_cores = 10
pool = Pool(number_of_cores)

Process ForkPoolWorker-29:
Process ForkPoolWorker-28:
Process ForkPoolWorker-25:
Process ForkPoolWorker-21:
Process ForkPoolWorker-30:
Process ForkPoolWorker-24:
Process ForkPoolWorker-23:
Process ForkPoolWorker-26:
Process ForkPoolWorker-22:
Traceback (most recent call last):
Process ForkPoolWorker-27:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/jana/anaconda3/envs/ri/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/home/jana/anaconda3/envs/ri/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/home/jana/anaconda3/envs/ri/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/home/jana/anacon

In [6]:
train = pd.DataFrame()
tweets = train_hass[0:10000] + train_counter[0:10000]
for row in tqdm(pool.imap_unordered(func=extract_information, 
                    iterable=tweets), total=len(tweets)):
        train = train.append(row, ignore_index=True)
#train.to_csv(join(dst, 'train_sample.csv.gzip'), index=False, compression='gzip')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20000/20000 [00:56<00:00, 351.66it/s]


In [7]:
test = pd.DataFrame()
tweets = test_hass[0:2000] + test_counter[0:2000]
for row in tqdm(pool.imap_unordered(func=extract_information, 
                    iterable=tweets), total=len(tweets)):
        test = test.append(row, ignore_index=True)
#test.to_csv(join(dst, 'test_sample.csv.gzip'), index=False, compression='gzip')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [00:08<00:00, 461.77it/s]


In [10]:
val = pd.DataFrame()
tweets = test_hass[2000:4000] + test_counter[2000:4000]
for row in tqdm(pool.imap_unordered(func=extract_information, 
                    iterable=tweets), total=len(tweets)):
        val = val.append(row, ignore_index=True)
#val.to_csv(join(dst, 'val_sample.csv.gzip'), index=False, compression='gzip')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [00:10<00:00, 383.17it/s]


In [28]:
for df, fname in zip([train, test, val], ['train', 'test', 'val']):
    df['label'] = df['label'].replace({'counter':0, 'hate':1})
    df = df.sample(frac=1, random_state=1).reset_index(drop=True)
    df['text'] = df['text'].apply(lambda x: x.replace('\n', ' '))
    np.savetxt(join(dst, '{}_text.txt'.format(fname)), df['text'].values, fmt='%s')
    np.savetxt(join(dst, '{}_labels.txt'.format(fname)), df['label'].values, fmt='%s')